In [37]:
import keras
import numpy as np

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32') / 255.
# x_train = np.pad(x_train, [(0,0), (14,14), (14,14), (0,0)], mode='constant', constant_values=0)
# y_train = keras.utils.to_categorical(y_train, num_classes=10)

x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test = x_test.astype('float32') / 255.
# x_test = np.pad(x_test, [(0,0), (14,14), (14,14), (0,0)], mode='constant', constant_values=0)
# y_test = keras.utils.to_categorical(y_test, num_classes=10)

In [38]:
import matplotlib.pyplot as plt
plt.imshow(x_train[0].reshape(56,56))

ValueError: cannot reshape array of size 784 into shape (56,56)

In [39]:
import tensorflow as tf
import numpy as np
import keras

def random_crop(x):
    cropped = tf.image.random_crop(x, size=[14, 14, 1])
    return cropped

class crop_generator(keras.utils.Sequence):
    def __init__(self, x_data, y_data, batch_size=32):
        self.x_data = x_data
        self.y_data = y_data
        self.batch_size = batch_size
        self.on_epoch_end()
        
    def __len__(self):
        return len(self.x_data) // self.batch_size
    
    def on_epoch_end(self):
        # create list of indices & shuffle
        self.indices = np.arange(len(self.x_data))
        np.random.shuffle(self.indices)
            
    def __getitem__(self, batch_index):
        data_indices = self.indices[batch_index*self.batch_size : (batch_index+1)*self.batch_size]
        X = np.empty((self.batch_size, 14, 14, 1))
#         X = np.empty((self.batch_size, 28,28, 1))
        Y = np.empty((self.batch_size), dtype=int)
        for i, idx in enumerate(data_indices):
            X[i] = random_crop(self.x_data[idx])
            Y[i] = self.y_data[idx]
        return X, keras.utils.to_categorical(Y, num_classes=10)

train_data = crop_generator(x_train, y_train)
test_data = crop_generator(x_test, y_test)
# for (x, y), i in zip(train_data, range(1)):
#     print(len(x), len(y))

In [42]:
from keras.models import Model
from keras.layers import Flatten, Conv2D, Conv2DTranspose, Dense, Input, Reshape, concatenate, Activation
from keras.utils import plot_model

# encoder
enc_input = Input(shape=(14,14,1), name='enc_input')
# enc_input = Input(shape=(56,56,1), name='enc_input')
# x = keras.layers.experimental.preprocessing.RandomCrop(14,14)(enc_input)
x = Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), activation='relu', padding='same', name='conv1')(enc_input)
x = Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), activation='relu', padding='same', name='conv2')(x)
x = Flatten(name='flatten')(x)
# latent = Dense(units=4, activation='sigmoid', name='latent')(x)
latent = Dense(units=4, name='latent')(x)

# classifier
# x = Dense(256, activation='relu')(latent)
# x = Dense(128, activation='relu')(x)
dec_input = Input(shape=(4,), name='dec_input')
x = Dense(64, activation='relu')(dec_input)
dec_output = Dense(10, activation='softmax', name='classifier_out')(x)

encoder = Model(enc_input, latent, name="Encoder")
decoder = Model(dec_input, dec_output, name="Decoder")
model = Model(encoder.input, decoder(encoder.output))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [43]:
history = model.fit(train_data, validation_data=(test_data), epochs=20)
# history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20)

Epoch 1/20
1875/1875 [==============================] - 47s 25ms/step - loss: 1.6882 - acc: 0.3900 - val_loss: 1.4265 - val_acc: 0.4766
Epoch 2/20
1875/1875 [==============================] - 48s 25ms/step - loss: 1.4094 - acc: 0.4958 - val_loss: 1.3129 - val_acc: 0.5287
Epoch 3/20
1875/1875 [==============================] - 47s 25ms/step - loss: 1.2999 - acc: 0.5401 - val_loss: 1.2444 - val_acc: 0.5623
Epoch 4/20
1875/1875 [==============================] - 47s 25ms/step - loss: 1.2395 - acc: 0.5635 - val_loss: 1.2019 - val_acc: 0.5738
Epoch 5/20
1875/1875 [==============================] - 45s 24ms/step - loss: 1.1959 - acc: 0.5797 - val_loss: 1.1356 - val_acc: 0.5936
Epoch 6/20
1875/1875 [==============================] - 45s 24ms/step - loss: 1.1670 - acc: 0.5883 - val_loss: 1.1150 - val_acc: 0.6096
Epoch 7/20
1875/1875 [==============================] - 45s 24ms/step - loss: 1.1367 - acc: 0.6001 - val_loss: 1.0946 - val_acc: 0.6188
Epoch 8/20
1875/1875 [==========================

In [22]:
# x = np.zeros((1,56,56,1), dtype='float32')
x = x_test[0].reshape(1,56,56,1)
pred = model.predict(x)
print(np.argmax(pred), np.max(pred))

7 0.99990845
